# NIfTI PET+mask mockup demo

Uses randomly-generated NIfTI PET volumes (and optional binary masks) to demonstrate the I/O pipeline and QCNN training.

The images are synthetic; predictive performance is not meaningful.

Outputs are written into:
- `Results/nifti_masked__results.csv`
- `Results/nifti_nomask__results.csv`

## Generate synthetic NIfTI mockups

In [ ]:
from pathlib import Path

!python examples/make_synthetic_nifti.py

print('masked:', Path('demo_data/nifti_masked').exists())
print('nomask:', Path('demo_data/nifti_nomask').exists())

## Train + evaluate QCNNs (no explanations)

In [ ]:
!python qnm_qai.py run \
  --input demo_data/nifti_masked \
  --input-type nifti \
  --methods pl_qcnn_alt,pl_qcnn_muw \
  --results-dir Results \
  --test-size 0.30 \
  --qcnn-epochs 2 \
  --qcnn-lr 0.05 \
  --max-samples-per-method 40 \
  --no-explain

## View summary metrics

In [ ]:
import pandas as pd
from pathlib import Path

summary = Path('Results')/'nifti_masked__results.csv'
pd.read_csv(summary)

## Inspect test predictions

In [ ]:
import pandas as pd
from pathlib import Path

base = Path('Results')/'nifti_masked'
for method in ['pl_qcnn_alt','pl_qcnn_muw']:
    p = base/method/'predictions'/'test.csv'
    print('\n', method, p)
    display(pd.read_csv(p).head(10))